In [1]:
import sys
import pathlib

sys.path.append(r"C:\Users\amrul\programming\deep_learning\dl_projects\Generative_Deep_Learning_2nd_Edition")

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import datasets, transforms
from tqdm import tqdm

IMAGE_SIZE = 64
BATCH_SIZE = 64
DATASET_REPETITIONS = 5
LOAD_MODEL = False

NOISE_EMBEDDING_SIZE = 32
PLOT_DIFFUSION_STEPS = 20

LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-4
EPOCHS = 25


In [2]:
checkpoints_folder = pathlib.Path.cwd()/"torch_checkpoints"
print(f"checkpoints folder : {checkpoints_folder} and it exists {checkpoints_folder.exists()}")
images_folder = pathlib.Path.cwd()/"output"/"generated_images_torch"
print(f"images folder : {images_folder} and it exists : {images_folder.exists()}")

checkpoints folder : c:\Users\amrul\programming\deep_learning\dl_projects\Generative_Deep_Learning_2nd_Edition\notebooks\08_diffusion\01_ddm\torch_checkpoints and it exists True
images folder : c:\Users\amrul\programming\deep_learning\dl_projects\Generative_Deep_Learning_2nd_Edition\notebooks\08_diffusion\01_ddm\output\generated_images_torch and it exists : True


In [3]:
from notebooks.utils import display

In [4]:
import torch
import torch.nn as nn
from ddm_torch_model_two import DiffusionModel,get_flower_images_train_dataset,convert_images_torch_to_numpy_for_display,training_loop,Normalizer

In [5]:
train_data = get_flower_images_train_dataset()
print(f"Loaded flower train dataset : {len(train_data)}")

Loaded flower train dataset : 6552


In [6]:
image,label = train_data[0]
print(f"image shape : {image.shape}")

image shape : torch.Size([3, 64, 64])


In [7]:
from torch.utils.data.sampler import Sampler
import numpy as np

class RepeatSampler(Sampler):
    def __init__(self, data_source, repetitions):
        self.data_source = data_source
        self.repetitions = repetitions

    def __iter__(self):
        n = len(self.data_source)

        # shuffle indices for each repetition
        indices = np.concatenate([np.random.permutation(n) for _ in range(self.repetitions)])
        #return iter(np.tile(np.arange(n), self.repetitions))
        return iter(indices)

    def __len__(self):
        return len(self.data_source) * self.repetitions


In [8]:
from torch.utils.data import DataLoader

repeat_sampler = RepeatSampler(train_data, DATASET_REPETITIONS)

# train_loader = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)
train_loader = DataLoader(
    train_data,
    batch_size=BATCH_SIZE,
    sampler=repeat_sampler    
)

ValueError: sampler option is mutually exclusive with shuffle

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [10]:
normalizer = Normalizer()

In [ ]:
# let's denormalize some images and display them
for idx,(images,_) in enumerate(train_loader):
    if idx>0:
        break
    display(convert_images_torch_to_numpy_for_display(images))
    display(convert_images_torch_to_numpy_for_display(normalizer.denormalize(images)))


In [12]:

mid_io_channels = [(96,128),(128,128)]
down_blocks_io_channels=[[(64,32),(32,32)],
                         [(32,64),(64,64)],
                         [(64,96),(96,96)],
                         ]

up_blocks_io_channels=[[(128,128),(224,96)],
                         [(192,96),(160,64)],
                         [(128,64),(96,32)]
                         ]
last_in_channels = 64
model = DiffusionModel(3,down_blocks_io_channels,up_blocks_io_channels,mid_io_channels,last_in_channels,normalizer,device)

if LOAD_MODEL:
    state_dict = torch.load(str(checkpoints_folder/"ddm_torch_checkpoints_31.pt"))
    model.load_state_dict(state_dict)

model = model.to(device)

In [13]:
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(),lr=LEARNING_RATE)

In [14]:
mae_loss = nn.L1Loss()

In [ ]:
ret = training_loop(EPOCHS,optimizer,model,mae_loss,train_loader,device,checkpoints_folder,images_folder)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(ret)